In [1]:
# All installations and checks
!export CUDA_VISIBLE_DEVICES=0,1,2,3
# !pip3 install transformers

In [2]:
# All imports
import argparse
import os
import sys
import shutil
import datetime
import torch.utils.data as torch_data
from util import *
from featurizer import *
from base_model import *
from torch_model import *
from evaluator import *

In [ ]:
# Global initializations and functions


In [3]:
# Create training data (i/o)

config = read_conf("wikisql_demo.conf")

featurizer = HydraFeaturizer(config)
train_data = SQLDataset(config["train_data_path"], config, featurizer, True)
train_data_loader = torch_data.DataLoader(train_data, batch_size=int(config["batch_size"]), shuffle=True, pin_memory=True)

num_samples = len(train_data)

tokenizer r dekho:  {'input_ids': [104, 31596, 24838, 105, 146789, 157958, 115, 1110, 11881, 116, 1110, 1198, 24892, 3399, 7906, 9778, 2107, 1154, 194438, 1115, 138, 105, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [4]:
# Creating the model and showing number of batches to be processed
note = ""
model_path = "model--"+datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
config["num_train_steps"] = int(num_samples * int(config["epochs"]) / int(config["batch_size"]))
step_per_epoch = num_samples / int(config["batch_size"])
print("total_steps: {0}, warm_up_steps: {1}".format(config["num_train_steps"], config["num_warmup_steps"]))

model = create_model(config, is_train=True)
evaluator = HydraEvaluator(model_path, config, featurizer, model, note)


total_steps: 0, warm_up_steps: 400


Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer r dekho:  {'input_ids': [104, 31596, 24838, 105, 146789, 157958, 115, 1110, 11881, 116, 1110, 1198, 24892, 3399, 7906, 9778, 2107, 1154, 194438, 1115, 138, 105, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [5]:
# Starting training
print("start training")
print(model_path)
loss_avg, step, epoch = 0.0, 0, 0
while True:
    for batch_id, batch in enumerate(train_data_loader):
        print(batch_id)
        cur_loss = model.train_on_batch(batch)
        loss_avg = (loss_avg * step + cur_loss) / (step + 1)
        step += 1
        print("here1")
        if batch_id % 1 == 0:
            print("here2")
            currentDT = datetime.datetime.now()
            print("[{3}] epoch {0}, batch {1}, batch_loss={2:.4f}".format(epoch, batch_id, cur_loss,
                                                                            currentDT.strftime("%m-%d %H:%M:%S")))
    model.save(model_path, epoch)
    evaluator.eval(epoch)
    epoch += 1
    if epoch >= int(config["epochs"]):
        break


start training
model--20221125_114000
0


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


pooled output before: 
torch.Size([6, 768])
bert output before: 
torch.Size([6, 96, 768])
here1
here2
[11-25 11:40:24] epoch 0, batch 0, batch_loss=10.4221
Model saved in path: model--20221125_114000/model_0.pt
model--20221125_114000/bad_cases
before_inf
model prediction start
pooled output before: 
torch.Size([6, 768])
bert output before: 
torch.Size([6, 96, 768])
model prediction end, time elapse: 0.9367790222167969
after_inf
wikidev.jsonl: overall:0.0, agg:0.0, sel:100.0, wn:0.0, wc:0.0, op:100.0, val:0.0
before_inf
model prediction start
pooled output before: 
torch.Size([6, 768])
bert output before: 
torch.Size([6, 96, 768])
model prediction end, time elapse: 0.6625607013702393
after_inf
wikitest.jsonl: overall:0.0, agg:0.0, sel:100.0, wn:0.0, wc:0.0, op:100.0, val:0.0
0
pooled output before: 
torch.Size([6, 768])
bert output before: 
torch.Size([6, 96, 768])
here1
here2
[11-25 11:40:34] epoch 1, batch 0, batch_loss=10.4293
Model saved in path: model--20221125_114000/model_1.pt
mo

In [ ]:
# Predicting SQL output of a single question input
config_demo = read_conf("wikisql_demo.conf")
featurizer_demo = HydraFeaturizer(config_demo)
model_path_demo = "model--20221123_123520"
epoch_demo = 0
model_demo = create_model(config_demo, is_train=True)
note = "_demo"
evaluator_demo = HydraEvaluator(model_path_demo, config_demo, featurizer_demo, model_demo, note)
model_demo.load(model_path_demo, epoch_demo)
evaluator_demo.eval(epoch_demo)